In [1]:
import pyspark 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local[*]') \
        .appName('test')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/11 03:22:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('code/data/pq/green/*/*')
df_yellow = spark.read.parquet('code/data/pq/yellow/*/*')

In [3]:
df_green.createOrReplaceTempView("green")
df_yellow.createOrReplaceTempView("yellow")

In [20]:
df_green_revenue = spark.sql("""
    SELECT 
        -- Revenue grouping 
        date_trunc('hour', lpep_pickup_datetime) AS hour, 
        PULocationID AS zone,
        
        -- Revenue calculation 
        SUM(total_amount) AS amount,
        count(1) as number_of_records
    FROM
        green
    WHERE lpep_pickup_datetime > '2020-01-01 00:00:00'
    GROUP BY
        1, 2
""")

In [17]:
df_green_revenue.show()

+-------------------+----+------------------+-----------------+
|               hour|zone|            amount|number_of_records|
+-------------------+----+------------------+-----------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|               45|
|2020-01-01 00:00:00|  17|195.03000000000003|                9|
|2020-01-01 00:00:00|  18|               7.8|                1|
|2020-01-01 00:00:00|  22|              15.8|                1|
|2020-01-01 00:00:00|  24|              87.6|                3|
|2020-01-01 00:00:00|  25| 531.0000000000002|               26|
|2020-01-01 00:00:00|  29|              61.3|                1|
|2020-01-01 00:00:00|  32| 68.94999999999999|                2|
|2020-01-01 00:00:00|  33|317.27000000000004|               11|
|2020-01-01 00:00:00|  35|            129.96|                5|
|2020-01-01 00:00:00|  36|295.34000000000003|               11|
|2020-01-01 00:00:00|  37|            175.67|                6|
|2020-01-01 00:00:00|  38| 98.7899999999

In [27]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('code/data/report/revenue/green', mode='overwrite')

In [24]:
df_yellow_revenue = spark.sql("""
    SELECT 
        -- Revenue grouping 
        date_trunc('hour', tpep_pickup_datetime) AS hour, 
        PULocationID AS zone,
        
        -- Revenue calculation 
        SUM(total_amount) AS amount,
        count(1) as number_of_records
    FROM
        yellow
    WHERE tpep_pickup_datetime > '2020-01-01 00:00:00'
    GROUP BY
        1, 2
""")

In [28]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('code/data/report/revenue/yellow', mode='overwrite')

In [31]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour','zone'], how='outer')

In [32]:
df_join.show()

+-------------------+----+------------------+-----------------+------------------+-----------------+
|               hour|zone|            amount|number_of_records|            amount|number_of_records|
+-------------------+----+------------------+-----------------+------------------+-----------------+
|2020-01-01 00:00:00|   3|              null|             null|              25.0|                1|
|2020-01-01 00:00:00|   4|              null|             null|1004.3000000000002|               57|
|2020-01-01 00:00:00|   7| 769.7299999999996|               45| 455.1700000000001|               38|
|2020-01-01 00:00:00|  12|              null|             null|             107.0|                6|
|2020-01-01 00:00:00|  37|            175.67|                6|161.60999999999999|                7|
|2020-01-01 00:00:00|  40|168.97999999999996|                8|             89.97|                5|
|2020-01-01 00:00:00|  45|              null|             null| 732.4800000000002|         

In [40]:
df_green_revenue = spark.read.parquet('code/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('code/data/report/revenue/yellow')

In [41]:
df_green_revenue_tmp = df_green_revenue \
 .withColumnRenamed('amount', 'green_amount') \
 .withColumnRenamed('number_of_records', 'green_number_of_records') 

df_yellow_revenue_tmp = df_yellow_revenue \
 .withColumnRenamed('amount', 'yellow_amount') \
 .withColumnRenamed('number_of_records', 'yellow_number_of_records') 

In [42]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [38]:
df_join \
.filter(df_join.zone == '17') \
.show()

+-------------------+----+------------------+-----------------------+-------------+------------------------+
|               hour|zone|      green_amount|green_number_of_records|yellow_amount|yellow_number_of_records|
+-------------------+----+------------------+-----------------------+-------------+------------------------+
|2020-01-01 01:00:00|  17| 598.1499999999999|                     18|       464.51|                      18|
|2020-01-01 06:00:00|  17|              16.3|                      1|         null|                    null|
|2020-01-01 09:00:00|  17|            123.25|                      3|         null|                    null|
|2020-01-01 12:00:00|  17|              27.6|                      2|         null|                    null|
|2020-01-01 22:00:00|  17|              16.6|                      2|         null|                    null|
|2020-01-02 00:00:00|  17|              null|                   null|         18.6|                       2|
|2020-01-02 07:00:0

In [43]:
df_join.write.parquet("code/data/report/revenue/total", mode='overwrite')

In [44]:
df_join = spark.read.parquet("code/data/report/revenue/total")

In [45]:
df_zones = spark.read.parquet("code/data/zones")

In [47]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [50]:
df_result.drop('LocationID').write.parquet('tmp/revenue-zones')

+-------------------+----+------------------+-----------------------+------------------+------------------------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|  Borough|                Zone|service_zone|
+-------------------+----+------------------+-----------------------+------------------+------------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              null|                   null|               8.8|                       1| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                   null|             34.09|                       1|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                      9|220.20999999999998|                       8| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00|  25| 531.0000000000002|       